In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import torch
import glob
import os
import pickle
from bs4 import BeautifulSoup
# data_path = '/Users/ruchiraravishankar/Downloads/iam_dataset/'
# data_path = '/content/drive/MyDrive/iam_dataset/'
data_path = '/content/drive/MyDrive/testing_proj/iam_dataset/'

# print(glob.glob(data_path))

In [ ]:
path = data_path + 'xml/*'
xml_path = glob.glob(path)
print(len(xml_path))

In [ ]:
# we only use d_i.pkl so no need to create d.pkl
# for file in xml_path:
# file = xml_path[0]
# print(file)
# d = {}

d_i = {}
def get_form_metadata(file):
    with open(file, 'r') as f:
        data = f.read()
    Bs_data = BeautifulSoup(data, "lxml")
    form = Bs_data.find('form')
    form_id = form.get('id')
    writer_id = form.get('writer-id')
#     print(form_id)
#     print(writer_id)
    # d[writer_id] = form_id
    d_i[form_id] = writer_id

for file in xml_path:
    get_form_metadata(file)
    

print(d_i)


In [ ]:
# we only use d_i.pkl so no need to create d.pkl
# a_file = open("/content/drive/MyDrive/iam_dataset/d.pkl", "wb")
# pickle.dump(d, a_file)
# a_file.close()

b_file = open(data_path + "d_i.pkl", "wb")
pickle.dump(d_i, b_file)
b_file.close()

In [ ]:
print(os.listdir(os.curdir))

In [ ]:

# a_file = open("/content/drive/MyDrive/iam_dataset/d.pkl", "rb")
# d = pickle.load(a_file)
# print(d)

b_file = open(data_path + "d_i.pkl", "rb")
d_i = pickle.load(b_file)
print(d_i)

In [ ]:
from collections import Counter

top_writers = []
num_writers = 50 # taking the top 50 writers
writers_counter = Counter(d_i.values())
for writer_id,_ in writers_counter.most_common(num_writers):
    top_writers.append(writer_id)

print(top_writers)

In [ ]:
top_forms = []
for form_id, author_id in d_i.items():
    if author_id in top_writers:
        top_forms.append(form_id)
print(top_forms)

In [ ]:
import os
import glob
import shutil

# Create temp directory to save writers' forms in (assumes files have already been copied if the directory exists)
temp_sentences_path = data_path + "temp_sentences"
if not os.path.exists(temp_sentences_path):
    os.makedirs(temp_sentences_path)
    # Copy forms that belong to the top 50 most common writers to the temp directory
    original_sentences_path = data_path + "sentences/**/**/*.png"
    for file_path in glob.glob(original_sentences_path):
        image_name = file_path.split('/')[-1]  
        file_name, _ = os.path.splitext(image_name)
        form_id = '-'.join(file_name.split('-')[0:2])
        if form_id in top_forms:
            shutil.copy2(file_path, temp_sentences_path + "/" + image_name)

In [ ]:
img_files = np.zeros((0), dtype=str)
print(img_files)
img_targets = np.zeros((0), dtype=str)
print(img_targets)
sentence_dict = {}
path_to_files = os.path.join(temp_sentences_path, '*')
for file_path in glob.glob(path_to_files):
    img_files = np.append(img_files, file_path)
    file_name, _ = os.path.splitext(file_path.split('/')[-1]  )
    form_id = '-'.join(file_name.split('-')[0:2])
    for key in d_i:
        if key == form_id:
            img_targets = np.append(img_targets, d_i[form_id])
#             sentence_dict[d_i[form_id]] = form_id

In [ ]:
print(img_files.shape)
print(img_targets.shape)

print(Counter(img_targets).keys())
print(Counter(img_targets).values())

In [ ]:
sen_dict = {}

for i in range(len(img_targets)):
    if img_targets[i] not in sen_dict.keys():
        sen_dict[img_targets[i]] = [img_files[i]]
    else:
        sen_dict[img_targets[i]].append(img_files[i])

In [ ]:
print(len(sen_dict['207']))

In [ ]:
# print(len(sen_dict.values()))
number_of = []
writer_ids = []
for key, value in sorted(sen_dict.items()):
    writer_ids.append(key)
    number_of.append(len([item for item in value if item]))
print(len(writer_ids))

In [ ]:
# crop so all images are of same size
# plot histogram 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.figure(figsize=(40,10))
plt.bar(writer_ids, number_of, width=0.5, color='g')
plt.show()

In [ ]:
for file_name in img_files[:2]:
    img = mpimg.imread(file_name)
    plt.figure(figsize = (10,10))
    plt.imshow(img, cmap ='gray')
    print(img.shape)

In [ ]:
writer_0_files = np.array(sen_dict['000'][:500])
print(writer_0_files.shape)

writer_0 = np.c_[writer_0_files, np.zeros(500)]

print(writer_0.shape)

In [ ]:
print(writer_0[0])

In [ ]:
labelled_data = np.stack([img_files, img_targets], axis=1)

print(labelled_data.shape)

In [ ]:
np.save(data_path + "labelled_data.npy", labelled_data)